import stuff

In [ ]:
#!pip install pyconll
#have to execute it cause not availbale in colab (just remove the #)

In [1]:
import pyconll
import numpy as np
import tensorflow as tf
import gc
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, TimeDistributed, Embedding, GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical
from collections import defaultdict
import re

# CONFIGURATION AND PATHS
TRAIN_PATH = 'fa_perdt-ud-train.conllu'
DEV_PATH = 'fa_perdt-ud-dev.conllu'
TEST_PATH = 'fa_perdt-ud-test.conllu'

# Hyperparameters
MAX_LEN = 100         # Sequence length (words per sentence)
# reduced this to 20 to reduce ram util
FEATURE_VECTOR_DIM = 20
#from 64 to 32
FEATURE_EMBEDDING_DIM = 32
EPOCHS = 5
BATCH_SIZE = 32

# Global feature dictionary for indexing features
feature_to_index = defaultdict(lambda: len(feature_to_index) + 1) # Start index at 1 for padding
# Reserve index 0 for padding

# Load
def load_conllu(path):
    """Loads CoNLL-U files and extracts sentences as lists of (words, tags)."""
    data = []
    try:
        conll = pyconll.load_from_file(path)
    except FileNotFoundError:
        print(f"Error: File not found at {path}. Returning empty data.")
        return []

    for sentence in conll:
        words = [token.form for token in sentence if token.form and token.upos]
        tags = [token.upos for token in sentence if token.form and token.upos]

        if len(words) == len(tags) and len(words) > 0:
            data.append((words, tags))
    return data

print("Loading data...")
train_data = load_conllu(TRAIN_PATH)
dev_data = load_conllu(DEV_PATH)
test_data = load_conllu(TEST_PATH)

X_train_list = [sent[0] for sent in train_data]
X_dev_list = [sent[0] for sent in dev_data]
X_test_list = [sent[0] for sent in test_data]




def extract_features_for_word(word):
    """
    MEMM model but this is as light as possible because all other models go out of availbale ram
    """
    features = set()
    word_lower = word.lower() #some kind of opt

    # Core lexical and morphological features
    features.add(f"word={word_lower}")
    features.add(f"prefix1={word_lower[:1]}")
    features.add(f"suffix2={word_lower[-2:]}")

    # Orthographic features
    if word.istitle():
        features.add("is_title")
    if word.isdigit():
        features.add("is_number")

    return features

def featurize_dataset(data_list):
    """Converts sentences into a 2D matrix of feature indices."""
    global feature_to_index

    all_sentences_features = []

    # 1. First pass (for training set only): Build feature vocabulary
    is_training_set = all(f in feature_to_index for f in extract_features_for_word(data_list[0][0])) == False
    if is_training_set:
        print("Building feature vocabulary...")
        temp_feature_to_index = defaultdict(lambda: len(temp_feature_to_index) + 1)
        # Use a flat list of all words for efficient vocabulary building
        all_words = [word for sentence in data_list for word in sentence]

        for word in all_words:
            features = extract_features_for_word(word)
            for f in features:
                 temp_feature_to_index[f] # Call defaultdict to populate it

        # Merge the temporary dictionary into the global one
        feature_to_index.update(temp_feature_to_index)
        print(f"Total unique features extracted: {len(feature_to_index)}")


    # 2. Second pass: Convert words into feature index vectors
    for sentence in data_list:
        sent_feature_indices = []

        for word in sentence:
            features = extract_features_for_word(word)

            # Convert features to indices (0 if not in vocabulary)
            indices = [feature_to_index[f] for f in features if f in feature_to_index]

            # Pad/truncate to FEATURE_VECTOR_DIM
            padded_indices = indices[:FEATURE_VECTOR_DIM]
            while len(padded_indices) < FEATURE_VECTOR_DIM:
                padded_indices.append(0) # 0 is the padding index

            sent_feature_indices.append(padded_indices)

        # Pad sentence to MAX_LEN
        while len(sent_feature_indices) < MAX_LEN:
            sent_feature_indices.append([0] * FEATURE_VECTOR_DIM)

        all_sentences_features.append(sent_feature_indices[:MAX_LEN])

    return np.array(all_sentences_features)

# --- Perform Feature Extraction ---
X_train_features_idx = featurize_dataset(X_train_list)
X_dev_features_idx = featurize_dataset(X_dev_list)
X_test_features_idx = featurize_dataset(X_test_list)


# --- STEP 2.9: PREPROCESSING (Tag Tokenization) ---

def preprocess_tags(data, max_len=MAX_LEN, tag_tokenizer=None):
    y_list = [sent[1] for sent in data]

    if tag_tokenizer is None:
        tag_tokenizer = Tokenizer(lower=False)
        tag_tokenizer.fit_on_texts(y_list)

    y_seq = tag_tokenizer.texts_to_sequences(y_list)
    y_padded = pad_sequences(y_seq, maxlen=max_len, padding='post', value=0)

    TAG_VOCAB_SIZE = len(tag_tokenizer.word_index) + 1
    y_categorical = to_categorical(y_padded, num_classes=TAG_VOCAB_SIZE)

    return y_categorical, y_padded, tag_tokenizer

y_train_cat, y_train_idx, tag_tokenizer = preprocess_tags(train_data)
y_dev_cat, y_dev_idx, _ = preprocess_tags(dev_data, tag_tokenizer=tag_tokenizer)
y_test_cat, y_test_idx, _ = preprocess_tags(test_data, tag_tokenizer=tag_tokenizer)

TAG_VOCAB_SIZE = len(tag_tokenizer.word_index) + 1

# --- Convert to TensorFlow Tensors and Clean Up NumPy Arrays ---
X_train_tf = tf.convert_to_tensor(X_train_features_idx, dtype=tf.int32)
y_train_tf = tf.convert_to_tensor(y_train_cat, dtype=tf.float32)

X_dev_tf = tf.convert_to_tensor(X_dev_features_idx, dtype=tf.int32)
y_dev_tf = tf.convert_to_tensor(y_dev_cat, dtype=tf.float32)

X_test_tf = tf.convert_to_tensor(X_test_features_idx, dtype=tf.int32)

# 🗑️ Explicitly clean up memory
del X_train_features_idx, X_dev_features_idx, X_test_features_idx
del y_train_cat, y_dev_cat
gc.collect()

print("\n--- SHAPE CONFIRMATION ---")
print(f"X_train_tf shape: {X_train_tf.shape} (Note the small feature dimension: {FEATURE_VECTOR_DIM})")

# --- Create TensorFlow Datasets for robust batching ---
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tf, y_train_tf)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
dev_dataset = tf.data.Dataset.from_tensor_slices((X_dev_tf, y_dev_tf)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


# --- STEP 3: MODEL DEFINITION (MEMM Approximation) ---

def create_memm_approximation_model(feature_vocab_size, tag_vocab_size, max_len, feature_dim, embed_dim):

    # Input is a 3D tensor of feature indices: (batch, MAX_LEN, FEATURE_VECTOR_DIM)
    input_layer = Input(shape=(max_len, feature_dim,), dtype='int32', name='feature_input')

    # 1. Feature Embedding Layer: Converts the sparse feature indices into a dense vector
    feature_embedding = TimeDistributed(Embedding(
        input_dim=feature_vocab_size,
        output_dim=embed_dim,       # Reduced output dimension (32)
        input_length=feature_dim,
        mask_zero=True
    ))(input_layer)

    # Global Average Pooling combines the feature embeddings for a single word
    dense_word_features = TimeDistributed(GlobalAveragePooling1D())(feature_embedding)

    # 2. Log-Linear Classifier (Approximation of MEMM)
    output_layer = TimeDistributed(Dense(tag_vocab_size, activation='softmax'))(dense_word_features)

    model = Model(inputs=input_layer, outputs=output_layer)

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    print(model.summary())
    return model

FEATURE_VOCAB_SIZE = len(feature_to_index) + 1
model = create_memm_approximation_model(FEATURE_VOCAB_SIZE, TAG_VOCAB_SIZE, MAX_LEN, FEATURE_VECTOR_DIM, FEATURE_EMBEDDING_DIM)

# --- STEP 4: TRAINING ---

print(f"\nTraining Lightweight MEMM Approximation Model for {EPOCHS} epochs...")
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=dev_dataset,
    verbose=1
)

# --- STEP 5: EVALUATION AND VISUALIZATION ---

def visualize_errors(original_sentences, y_true_padded_idx, y_pred_padded_idx, tag_tokenizer, max_samples=5):
    """Finds and prints up to max_samples sentences with incorrect predictions."""

    idx_to_tag = {v: k for k, v in tag_tokenizer.word_index.items()}
    error_count = 0

    print("\n\n--- Sample Incorrect Predictions ---")

    for i in range(len(original_sentences)):
        true_tags = []
        pred_tags = []
        words = original_sentences[i]

        for j in range(len(words)):
            true_idx = y_true_padded_idx[i, j]
            pred_idx = y_pred_padded_idx[i, j]

            if true_idx != 0:
                true_tags.append(idx_to_tag.get(true_idx, 'UNK'))
                pred_tags.append(idx_to_tag.get(pred_idx, 'UNK'))

        has_error = any(t != p for t, p in zip(true_tags, pred_tags))

        if has_error and error_count < max_samples:
            error_count += 1
            print(f"\n[ERROR SENTENCE {error_count}]")

            print("  {:<15} {:<15} {:<15}".format("WORD", "TRUE TAG", "PREDICTED TAG"))
            print("  " + "-"*45)

            for word, true_tag, pred_tag in zip(words, true_tags, pred_tags):
                prediction_status = "❌" if true_tag != pred_tag else "✓"
                print(f"{prediction_status} " + "{:<15} {:<15} {:<15}".format(word, true_tag, pred_tag))

def evaluate_model(model, X_data_tf, y_true_padded_idx, tag_tokenizer, original_data_list, dataset_name='set'):
    """Performs tagging and prints metrics, including error visualization."""

    y_pred_proba = model.predict(X_data_tf, verbose=0)

    y_pred_padded_idx = np.argmax(y_pred_proba, axis=-1)

    y_true = []
    y_pred = []

    for true_seq, pred_seq in zip(y_true_padded_idx, y_pred_padded_idx):
        for true_idx, pred_idx in zip(true_seq, pred_seq):
            if true_idx != 0:
                y_true.append(tag_tokenizer.index_word.get(true_idx, 'UNK'))
                y_pred.append(tag_tokenizer.index_word.get(pred_idx, 'UNK'))

    print(f'\n{"="*10} Evaluation on {dataset_name} Set {"="*10}')

    if not y_true:
        print("No valid data for evaluation.")
        return

    print('Overall Tagging Accuracy:', accuracy_score(y_true, y_pred))

    print('\nDetailed Metrics (Precision, Recall, F1-Score) per UPOS Tag:')

    target_names = sorted(list(set(y_true)))
    print(classification_report(y_true, y_pred, labels=target_names, target_names=target_names, digits=4, zero_division=0))
    print('='*50)

    visualize_errors(original_data_list, y_true_padded_idx, y_pred_padded_idx, tag_tokenizer)


# Call the functions

# 1. Evaluate on Development Set
evaluate_model(model, X_dev_tf, y_dev_idx, tag_tokenizer, X_dev_list, 'Development (Dev)')

# 2. Evaluate on Test Set
evaluate_model(model, X_test_tf, y_test_idx, tag_tokenizer, X_test_list, 'Final Test')

Loading data...
Building feature vocabulary...
Total unique features extracted: 33139

--- SHAPE CONFIRMATION ---
X_train_tf shape: (26196, 100, 20) (Note the small feature dimension: 20)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ feature_input (InputLayer)      │ (None, 100, 20)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 100, 20, 32)    │     1,060,480 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 100, 32)        │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 100, 17)        │           561 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,061,041 (4.05 MB)

 Trainable params: 1,061,041 (4.05 MB)

 Non-trainable params: 0 (0.00 B)

None

Training Lightweight MEMM Approximation Model for 5 epochs...
Epoch 1/5
819/819 ━━━━━━━━━━━━━━━━━━━━ 321s 239ms/step - accuracy: 0.8252 - loss: 0.7354 - val_accuracy: 0.9006 - val_loss: 0.3065
Epoch 2/5
819/819 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.9243 - loss: 0.2614 - val_accuracy: 0.9597 - val_loss: 0.1499
Epoch 3/5
819/819 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.9594 - loss: 0.1441 - val_accuracy: 0.9672 - val_loss: 0.1093
Epoch 4/5
819/819 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step - accuracy: 0.9671 - loss: 0.1084 - val_accuracy: 0.9725 - val_loss: 0.0897
Epoch 5/5
819/819 ━━━━━━━━━━━━━━━━━━━━ 14s 18ms/step - accuracy: 0.9723 - loss: 0.0897 - val_accuracy: 0.9757 - val_loss: 0.0773

========== Evaluation on Development (Dev) Set ==========
Overall Tagging Accuracy: 0.8589390492156628

Detailed Metrics (Precision, Recall, F1-Score) per UPOS Tag:
              precision    recall  f1-score   support

         ADJ     0.7333    0.3937    0.5123      1872
         A

IndexError: index 100 is out of bounds for axis 1 with size 100